# Usage: scenario analysis
This is a quick tour of CovsirPhy. Details scenario analysis will be explained.
"Scenario analysis" means that we calculate the number of cases in the future phases with some sets of ODE parameter values. With this analysis, we can estimate the impact of our activities against the outbreak on the number of cases.

### Preparation
Prepare the packages.

In [ ]:
from pprint import pprint

In [ ]:
import covsirphy as cs
cs.__version__

### Dataset preparation
Download the datasets to "../input" directory and load them.  
Please refer to [Usage: datasets](https://lisphilar.github.io/covid19-sir/usage_dataset.html) for the details.

In [ ]:
data_loader = cs.DataLoader("../input")
# The number of cases (JHU style)
jhu_data = data_loader.jhu()
# Population in each country
population_data = data_loader.population()
# Government Response Tracker (OxCGRT)
oxcgrt_data = data_loader.oxcgrt()
# The number of tests
pcr_data = data_loader.pcr()
# The number of vaccinations
vaccine_data = data_loader.vaccine()

### Start scenario analysis
As an example, we will analysis the number of cases in Japan. `covsirphy.Scenario` is the interface for analysis. Please specify the area (country: required, province: optional) and register the datasets. From version 2.17.0, we use `Scenario.register()` method to register datasets.

In [ ]:
# Specify country and province (optinal) names
snl = cs.Scenario(country="Japan", province=None)
# Register datasets
snl.register(jhu_data, population_data, extras=[oxcgrt_data, pcr_data, vaccine_data])

We call `JHUData` and `PopulationData` as "the main datasets" because they are required to calculate the number of susceptible/infected/recovered/fatal cases. These variables are used in SIR-F model.  
The other datasets are called as "the extra datasets" and they will be used to predict the futute parameter values of SIR-F model for forecasting the number of cases with some scenarios.

Additional information:  

- Details of the datasets: [Usage: datasets](https://lisphilar.github.io/covid19-sir/usage_dataset.html)
- Details of SIR-F model: [Usage: SIR-derived models](https://lisphilar.github.io/covid19-sir/usage_theoretical.html)

#### Display/save figures
We have intactive mode and script mode to display/save figures.

Interactive mode:  
This is recommended when we use interactive shells, including Jupyter Notebook, for analysis. Figures will be displayed.

In [ ]:
# Show figures (default is True when we use interactive shells)
snl.interactive = True
# If you do not want to display figures for selected methods, please apply "show_figures=False"
# snl.records(show_figure=False)

Script mode:  
This will be forced when we use CovsirPhy with scripts. Figures will be shown when filename is specified.

In [ ]:
# Stop displaying figures
# snl.interactive = False
# With this mode we can save figures with "filename" argument
# snl.records(filename="records.jpg")

### Check records
Let's see the records at first. `Scenario.records()` method return the records as a pandas dataframe and show a line plot. Some kind of complement will be done for analysis, if necessary.

`Scenario.records()` shows the number of infected/recovered/fatal cases as default. Using `variables` argument, we can set the variables to show.

In [ ]:
# Show main variables
snl.records().tail()
# We can select variables to show
# snl.records(variables=["Confirmed", "Recovered"])

All available variables can be retrieved with `variables="all"`.

In [ ]:
# All available variables for the registered datasets
df = snl.records(variables="all", show_figure=False)
pprint(df.set_index("Date").columns.tolist(), compact=True)

In [ ]:
# We can select the varialbles of both of the main and extra datasets if registered
snl.records(variables=["Tests", "Vaccinations"]).tail()

We can calculate the number of daily new cases with `Scenario.record_diff()` method.

In [ ]:
# Acceptable variables are the same as Scenario.records()
_ = snl.records_diff(variables=["Confirmed"], window=7)

`Scenario.show_complement()` method is useful to show the kinds of complement. The details of complement are explained in [Usage: datasets](https://lisphilar.github.io/covid19-sir/usage_dataset.html#The-number-of-cases-(JHU-style)) section.

In [ ]:
# Show the details of complement
snl.show_complement()

### S-R trend analysis
S-R trend analysis finds the change points of SIR-derived ODE parameters. Details will be explained in [Usage (details: phases)](https://lisphilar.github.io/covid19-sir/usage_phases.html). Phases will be separated with dotted lines.

In [ ]:
# Perform S-R trend analysis and set phases
_ = snl.trend()
# Show the summary of phases
snl.summary()
# Or,
# snl.trend().summary()

### Hyperparameter estimation of ODE models
Here, we will estimate the parameter values of SIR-derived models. As an example, we use SIR-F model. Details of models will be explained in [Usage (details: theoritical datasets)](https://lisphilar.github.io/covid19-sir/usage_theoretical.html).  

**We can select the model from SIR, SIRD and SIR-F model for parameter estimation. SIR-FV model (completely deprecated) and SEWIR-F model cannot be used.**

In [ ]:
# Estimate the parameter values of SIR-F model
# Default value of timeout is 180 sec
snl.estimate(cs.SIRF, timeout=180)

In [ ]:
# Show the summary of parameter estimation
snl.summary()

### Evaluation of estimation accuracy
Accuracy of parameter estimation can be evaluated with RMSLE (Root Mean Squared Log Error) score.  

\begin{align*}
\mathrm{RMSLE} = \sqrt{\cfrac{1}{n}\sum_{i=1}^{n}(log_{10}(A_{i} + 1) - log_{10}(P_{i} + 1))^2}
\end{align*}

Where $A$ is the observed (actual) values, $P$ is estimated (predicted) values. Variables are $S (i=1), I (i=2), R (i=3)\ \mathrm{and}\ F (i=n=4)$ for SIR-F model. When RMSLE score is low, hyperparameter estimation is highly accurate.
Please refer to external sites, including [Medium: What’s the Difference Between RMSE and RMSLE?](https://medium.com/analytics-vidhya/root-mean-square-log-error-rmse-vs-rmlse-935c6cc1802a)

In [ ]:
# Show RMSLE scores with the number of optimization trials and runtime for phases
snl.summary(columns=["Start", "End", "RMSLE", "Trials", "Runtime"])

Additionally, we can visualize the accuracy with `Scenario.estimate_accuracy()`, specifing phase name.

In [ ]:
# Visualize the accuracy for the 2nd phase
snl.estimate_accuracy(phase="2nd")
# phase="last" means the last phases
# snl.estimate_accuracy(phase="last")

Total score can be retrived with `Scenario.score()` method.

In [ ]:
# Get total score: metrics="MAE", "MSE", "MSLE", "RMSE" or "RMSLE"
# snl.score(metrics="RMSLE")
metrics_list = ["MAE", "MSE", "MSLE", "RMSE", "RMSLE"]
for metrics in metrics_list:
    metrics_name = metrics.rjust(len(max(metrics_list, key=len)))
    print(f"{metrics_name}: {snl.score(metrics=metrics):.3f}")

### Get parameter value
We can get the parameter values of a phase using `Scenario.get()` method.

In [ ]:
# Get parameter values
snl.get("Rt", phase="4th")

In [ ]:
# phase="last" means the last phases
snl.get("Rt", phase="last")

### Show parameter history
We can get the history of parameter values with a dataframe and a figure.

In [ ]:
# Get the parameter values as a dataframe
snl.summary(columns=[*cs.SIRF.PARAMETERS, "Rt"])

`Scenario.history()` method shows the trajectories of parameters (and the number of cases).

In [ ]:
_ = snl.history(target="theta", show_legend=False)

In [ ]:
_ = snl.history(target="kappa", show_legend=False)

In [ ]:
_ = snl.history(target="rho", show_legend=False)

In [ ]:
_ = snl.history(target="sigma", show_legend=False)

Notes on the history of $\sigma$ value in japan (last updated: 28Dec2020):  
In Japan, we experienced two waves and we are in third wave. In the first wave (Apr - May), recovery period was too long because collapse of the medical care system occurred and no medicines were found.

Sigma values: the first wave < the second wave > the third wave

However, in the second wave (Jul - Oct), recovery period appears short because we have some effective medicines (not approved, in clinical study), yonger people (people un-associated to sever diseases) were infected.

In the third wave (Nov - ), older people tend to be infected and we are facing with medical collapse at this time...

### Show the history of reproduction number
$R_0$ ("R naught") means "the average number of secondary infections caused by an infected host" ([Infection Modeling — Part 1](https://towardsdatascience.com/infection-modeling-part-1-87e74645568a)). When this value is larger than 1, the infection disease is outbreaking.

In [ ]:
_ = snl.history(target="Rt", show_legend=False)

### Simulate the number of cases
We can compare the actual and simulated (with estimated parameter values) number of confirmed/infected/recovered/fatal cases using `Scenario.history()` method.

In [ ]:
# Compare the actual values and the main scenario
_ = snl.history("Infected")

When we want to show only one scenario with all variables, we use `Scenario.simulate()` method.

In [ ]:
# Default variables and all phases for main scenario
_ =snl.simulate(name="Main")

In [ ]:
# We canselect variables and phases
_ = snl.simulate(name="Main", variables=["Confirmed", "Infected"], phases=["5th", "6th"])

### Main scenario
To investigate the effect of parameter changes, we will perform scenario analysis. In the main scenario, we will assume that the parameter values do not change after the last past phase.

i.e. If the parameter velues will not be changed until 01Jun2021, how many cases will be? We call this scenario as "Main" scenario.

In [ ]:
# Clear future phases in Main scenario
snl.clear(name="Main")
# Add one future phase 30 days with the parameter set of the last past phase
snl.add(days=30, name="Main")
# Add one future phase until 01Apr2021 with the same parameter set
snl.add(end_date="01Jun2021", name="Main")
# Simulate the number of cases
snl.simulate(name="Main").tail()

### Medicine scenario
To investigate the effect of new medicines, we will assume that $\sigma$ will be changed in the future phases.

If $\sigma$ will be 1.2 times in 30 days, how many cases will be? We will call this scenario as "Medicine" scenario.

In [ ]:
# Calcuate the current sigma value of the last phase
sigma_current = snl.get("sigma", name="Main", phase="last")
sigma_current

In [ ]:
# Sigma value will be double
sigma_new = sigma_current * 1.2
sigma_new

In [ ]:
# Initialize "Medicine" scenario (with the same past phases as that of Main scenario)
snl.clear(name="Medicine")
# Add 30 days as a new future phases with the same parameter set
snl.add(name="Medicine", days=30, sigma=sigma_current)
# Add a phase until 01Jun2021 with doubled sigma value
snl.add(name="Medicine", end_date="01Jun2021", sigma=sigma_new)
snl.summary(name="Medicine")

In [ ]:
# Simulate the number of cases
_ = snl.simulate(name="Medicine").tail()

In [ ]:
# Compare the number of cases with main scenario for the last two phases
all_phases = snl.summary(name="Medicine").index.tolist()
# Should start with a past phase: the last phase, the first and the second future phase
_ = snl.history("Infected", phases=all_phases[-3:])

### Short-term prediction of parameter values
With extra datasets, we can predict the parameter values of the future phases because [OxCGRT indicators](https://github.com/OxCGRT/covid-policy-tracker) (policy measures), vaccinations and so on impact on parameter values with the delay period. Delay period will be calculated with `cenario.estimate_delay()` automatically.

OxCGRT indicators are

- school_closing,
- workplace_closing,
- cancel_events, 
- gatherings_restrictions,
- transport_closing,
- stay_home_restrictions,
- internal_movement_restrictions,
- international_movement_restrictions,
- information_campaigns,
- testing_policy, and
- contact_tracing.

In [ ]:
# Create Forecast scenario (copy Main scenario and delete future phases)
snl.clear(name="Forecast", template="Main")
# Fitting with linear regression model (Elastic Net regression)
fit_dict = snl.fit(name="Forecast")
print(f"Determination coefficient: {fit_dict['score_train']:.3f} (train)")
print(f"Determination coefficient: {fit_dict['score_test']:.3f} (test)")
print("Intercept:")
fit_dict["intercept"].style.background_gradient(axis=None)

In [ ]:
# Short-term prediction
snl.predict(name="Forecast").summary(name="Forecast")

In [ ]:
# Or, when you do not need 'fit_dict',
# snl.fit_predict(name="Forecast").summary(name="Forecast")

In [ ]:
# Set 01Jun2021 as tthe last date of the future
snl.add(name="Forecast", end_date="01Jun2021")
# Simulate the number of cases
_ = snl.simulate(name="Forecast").tail()

In [ ]:
# Compare the number of cases with main scenario for the last two phases
all_phases = snl.summary(name="Forecast").index.tolist()
# Should start with a past phase: the last phase, the first and the second future phase
_ = snl.history("Infected", phases=all_phases[-3:])

### Compare the scenarios
We will compare the scenarios with representative values, reproduction number and parameter values. Currently, we can compare the scenarios with the following indexes.

- max(Infected): max value of Infected
- argmax(Infected): the date when Infected shows max value
- Infected on …: Infected on the end date of the last phase
- Fatal on …: Fatal on the end date of the last phase

In [ ]:
snl.describe()

In [ ]:
_ = snl.history(target="Infected")

In [ ]:
_ = snl.history(target="Rt")

In [ ]:
_ = snl.history(target="rho")

In [ ]:
_ = snl.history(target="sigma")

In [ ]:
_ = snl.history(target="theta")

In [ ]:
_ = snl.history(target="kappa")

### Change rate of parameters in main scenario
History of each parameter will be shown. Values will be divided by the values in 0th phase.

In [ ]:
_ = snl.history_rate(name="Main")

## Retrospective analysis
We can evaluate the impact of measures using past records. How many people were infected if the parameter values have not changed sinse 01Sep2020?

In [ ]:
# Perform retrospective analysis
snl_retro = cs.Scenario(jhu_data, population_data, "Japan")
snl_retro.retrospective(
    "01Jan2021", model=cs.SIRF, control="Main", target="Retrospective", timeout=10)

In [ ]:
# Show the summary of estimation
cols = ["Start", "End", "ODE", "Rt", *cs.SIRF.PARAMETERS] + ["RMSLE", "Trials", "Runtime"]
snl_retro.summary(columns=cols)

In [ ]:
# History of reproduction number
_ = snl_retro.history("Rt")

In [ ]:
# History of Infected
_ = snl_retro.history("Infected")

In [ ]:
# Show the representative values
snl_retro.describe()